In [2]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
d_path = '/kaggle/input'
# d_path = '.'

trn_dirs = glob(f'{d_path}/understanding_cloud_organization/train_images/*')
tst_dirs = glob(f'{d_path}/understanding_cloud_organization/test_images/*')

In [4]:
print(tst_dirs)

['/kaggle/input/understanding_cloud_organization/test_images/cb6c11f.jpg', '/kaggle/input/understanding_cloud_organization/test_images/efe623a.jpg', '/kaggle/input/understanding_cloud_organization/test_images/80688f9.jpg', '/kaggle/input/understanding_cloud_organization/test_images/96140c9.jpg', '/kaggle/input/understanding_cloud_organization/test_images/7e13cee.jpg', '/kaggle/input/understanding_cloud_organization/test_images/f6379c3.jpg', '/kaggle/input/understanding_cloud_organization/test_images/af0fd17.jpg', '/kaggle/input/understanding_cloud_organization/test_images/1d6677f.jpg', '/kaggle/input/understanding_cloud_organization/test_images/fa46c0b.jpg', '/kaggle/input/understanding_cloud_organization/test_images/ed1b901.jpg', '/kaggle/input/understanding_cloud_organization/test_images/7bfb4bf.jpg', '/kaggle/input/understanding_cloud_organization/test_images/57eb605.jpg', '/kaggle/input/understanding_cloud_organization/test_images/aaccdf8.jpg', '/kaggle/input/understanding_cloud_or

In [5]:
trn_df = pd.read_csv(f'{d_path}/understanding_cloud_organization/train.csv')
trn_df

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...
...,...,...
22179,ffd6680.jpg_Sugar,NaN
22180,ffea4f4.jpg_Fish,NaN
22181,ffea4f4.jpg_Flower,1194860 675 1196260 675 1197660 675 1199060 67...
22182,ffea4f4.jpg_Gravel,NaN


In [6]:
trn_df[['Image','Label']] = trn_df['Image_Label'].str.split('_', expand=True)
trn_df.drop('Image_Label', axis=1, inplace=True)
trn_df

,EncodedPixels,Image,Label
0,264918 937 266318 937 267718 937 269118 937 27...,0011165.jpg,Fish
1,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165.jpg,Flower
2,NaN,0011165.jpg,Gravel
3,NaN,0011165.jpg,Sugar
4,233813 878 235213 878 236613 878 238010 881 23...,002be4f.jpg,Fish
...,...,...,...
22179,NaN,ffd6680.jpg,Sugar
22180,NaN,ffea4f4.jpg,Fish
22181,1194860 675 1196260 675 1197660 675 1199060 67...,ffea4f4.jpg,Flower
22182,NaN,ffea4f4.jpg,Gravel


In [7]:
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms.functional as F
import torch

class CustomDataset(Dataset):
    def __init__(self, data_dirs, df, transform=None, image_size=(1400, 2100), target_size=(350, 525), pad_size=(352, 544)):
        self.data_dirs = data_dirs
        self.df = df
        self.transform = transform
        self.image_size = image_size
        self.target_size = target_size  # 마스크와 이미지를 리사이즈할 타겟 사이즈 (350, 525)
        self.pad_size = pad_size  # UNet을 위해 패딩 후의 크기 (352, 544)
        
    def __len__(self):
        return len(self.data_dirs)
    
    def resize_and_pad_image_and_mask(self, image, masks):
        """이미지와 마스크를 리사이즈한 후 패딩"""
        # 1. 이미지와 마스크를 350x525로 리사이즈
        resized_image = F.resize(image, self.target_size)
        resized_masks = [F.resize(Image.fromarray(mask), self.target_size, interpolation=Image.NEAREST) for mask in masks]

        # 2. 352x544로 패딩 추가 (오른쪽과 아래쪽에 패딩 추가)
        pad_width = self.pad_size[1] - self.target_size[1]  # (544 - 525)
        pad_height = self.pad_size[0] - self.target_size[0]  # (352 - 350)
        padding = (0, 0, pad_width, pad_height)  # (left, top, right, bottom) 패딩 설정

        padded_image = F.pad(resized_image, padding, fill=0)  # 이미지: 검정색 패딩 추가
        padded_masks = [F.pad(mask, padding, fill=0) for mask in resized_masks]  # 마스크: 0으로 패딩 추가

        return padded_image, padded_masks
    
    def __getitem__(self, idx):
        img_path = self.data_dirs[idx]
        img_name = img_path.split('/')[-1]
        
        img = Image.open(img_path)
        obj_df = self.df.loc[self.df['Image'] == img_name]
        masks = []
        for i in range(len(obj_df)):
            encoded = obj_df['EncodedPixels'].iloc[i]
            if pd.isna(encoded):
                masks.append(np.zeros((1400, 2100), dtype=np.uint8))  # 마스크 배열을 명시적으로 uint8로 설정
            else:
                pixels = encoded.split()
                mask = np.zeros(1400 * 2100, dtype=np.uint8)  # 마스크 배열을 명시적으로 uint8로 설정
                for i in range(0, len(pixels), 2):
                    pixel = int(pixels[i])
                    cnt = int(pixels[i+1])
                    mask[pixel:pixel+cnt] = 1.0
                masks.append(mask.reshape((1400, 2100)).T)

        padded_image, padded_masks = self.resize_and_pad_image_and_mask(img, masks)

        if self.transform:
            padded_image = self.transform(padded_image)
        
        padded_masks = [np.array(mask) for mask in padded_masks]  # 마스크를 NumPy 배열로 변환
        padded_masks = np.stack(padded_masks, axis=0)  # (4, 352, 544)의 형태로 쌓기
        padded_masks = torch.from_numpy(padded_masks).float()
        
        return padded_image, padded_masks

In [8]:
from torchvision import transforms

# 이미지 전처리를 위한 Transform 정의
transform = transforms.Compose([
#     transforms.Resize((350, 525)),
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

In [9]:
sky_dataset = CustomDataset(trn_dirs, trn_df, transform=transform)
print(sky_dataset.__len__())

train_size = int(0.8 * len(sky_dataset))
val_size = len(sky_dataset) - train_size

train_ds, val_ds = random_split(sky_dataset, [train_size, val_size])
print(f"Train dataset size: {len(train_ds)} | Validation dataset size: {len(val_ds)}")

train_loader = DataLoader(train_ds, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_ds, batch_size = 32, shuffle = False)

imgs, masks = next(iter(train_loader))
print(imgs.shape, masks.shape)

5546
Train dataset size: 4436 | Validation dataset size: 1110
torch.Size([32, 3, 352, 544]) torch.Size([32, 4, 352, 544])


In [10]:
!pip install segmentation-models-pytorch -q

In [19]:
import segmentation_models_pytorch as smp

device = 'cuda' if torch.cuda.is_available() else 'cpu'
unet = smp.Unet('resnet34', encoder_weights='imagenet', in_channels=3, classes=4)
unet.to(device)
unet

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [20]:
mode = 'multilabel'
criterion = smp.losses.JaccardLoss(mode, classes=4)
optim = torch.optim.Adam(params = unet.parameters(), lr = 5e-4)
epochs = 10

In [21]:
def metrics(pred, target, mode, threshold):
#     pred = pred.round().long()
    target = target.round().long()
    
    tp, fp, fn, tn = smp.metrics.get_stats(pred, target, mode, threshold=threshold)
    
    iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction='macro')
    f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction='macro')
    accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction='macro')
    
    return iou_score, f1_score, accuracy

In [22]:
dummy_input = torch.randn(1, 3, 352, 544).to(device)
dummy_output = unet(dummy_input)
dummy_output.shape

torch.Size([1, 4, 352, 544])

In [23]:
dummy_gt = torch.rand(1, 4, 352, 544).to(device)
# num_classes: mode = 'multiclass'일 경우에만 필요
# threshold: mode = 'binary', 'multilabel'일 경우에만 필요
iou, dice, acc = metrics(dummy_output, dummy_gt, mode, 0.5)
iou, dice, acc

(tensor(0.0908, device='cuda:0'),
 tensor(0.1522, device='cuda:0'),
 tensor(0.5001, device='cuda:0'))

In [24]:
from tqdm.notebook import tqdm

def train_and_validate(model, train_loader, val_loader, optim, criterion, epochs, num_classes=4, save_path='best_model.pth'):
    train_losses = []
    train_IoUs = []
    train_Dices = []
    train_PAs = []
    
    val_losses = []
    val_IoUs = []
    val_Dices = []
    val_PAs = []
    
    best_val_loss = float('inf')  # 검증 손실을 추적하기 위한 변수
    
    for epoch in range(epochs):
        train_loss = 0
        train_IoU = 0
        train_Dice = 0
        train_PA = 0
        model.train()
        for img, mask in tqdm(train_loader):
            img = img.to(device)
            mask = mask.to(device)
            optim.zero_grad()
            output = model(img)
            loss = criterion(output, mask)
            iou, dice, pa = metrics(output, mask, mode, 0.5)
            loss.backward()
            optim.step()
            train_loss += loss.item()
            train_IoU += iou
            train_Dice += dice
            train_PA += pa
            
        print(f"Epoch {epoch + 1}\nTrain loss: {train_loss / len(train_loader):.2f} | Train IoU: {train_IoU / len(train_loader):.2f} | Train Dice: {train_Dice / len(train_loader):.2f} | Train Pixel Acc: {train_PA / len(train_loader):.2f}")
        train_losses.append(train_loss / len(train_loader))
        train_IoUs.append(train_IoU / len(train_loader))
        train_Dices.append(train_Dice / len(train_loader))
        train_PAs.append(train_PA / len(train_loader))
        
        val_loss = 0
        val_IoU = 0
        val_Dice = 0
        val_PA = 0
        model.eval()
        with torch.no_grad():
            for img, mask in tqdm(val_loader):
                img = img.to(device)
                mask = mask.to(device)
                output = model(img)
                loss = criterion(output, mask)
                iou, dice, pa = metrics(output, mask, mode, 0.5)
                val_loss += loss.item()
                val_IoU += iou
                val_Dice += dice
                val_PA += pa
                
            print(f"Validation loss: {val_loss / len(val_loader):.2f} | Validation IoU: {val_IoU / len(val_loader):.2f} | Validation Dice: {val_Dice / len(val_loader):.2f} | Validation Pixel Acc: {val_PA / len(val_loader):.2f}\n{'='*100}")
            val_losses.append(val_loss / len(val_loader))
            val_IoUs.append(val_IoU / len(val_loader))
            val_Dices.append(val_Dice / len(val_loader))
            val_PAs.append(val_PA / len(val_loader))
            
        # 베스트 성능일 때 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved at epoch {epoch + 1} with validation loss {best_val_loss:.4f}")
            
    return train_losses, train_IoUs, train_Dices, train_PAs, val_losses, val_IoUs, val_Dices, val_PAs



In [25]:
train_losses, train_IoUs, train_Dices, train_PAs, val_losses, val_IoUs, val_Dices, val_PAs = train_and_validate(unet, train_loader, val_loader, optim, criterion, epochs, 4)

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 1
Train loss: 0.76 | Train IoU: 0.26 | Train Dice: 0.41 | Train Pixel Acc: 0.75


  0%|          | 0/35 [00:00<?, ?it/s]

Validation loss: 0.75 | Validation IoU: 0.26 | Validation Dice: 0.41 | Validation Pixel Acc: 0.76
Best model saved at epoch 1 with validation loss 26.2060


  0%|          | 0/139 [00:00<?, ?it/s]

...

Validation loss: 0.68 | Validation IoU: 0.32 | Validation Dice: 0.47 | Validation Pixel Acc: 0.83
Best model saved at epoch 9 with validation loss 23.9521


  0%|          | 0/139 [00:00<?, ?it/s]

Epoch 10
Train loss: 0.63 | Train IoU: 0.37 | Train Dice: 0.53 | Train Pixel Acc: 0.84


  0%|          | 0/35 [00:00<?, ?it/s]

Validation loss: 0.74 | Validation IoU: 0.26 | Validation Dice: 0.41 | Validation Pixel Acc: 0.82


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

class TestDataset(Dataset):
    def __init__(self, image_dirs, transform=None, target_size=(350, 525), pad_size=(352, 544)):
        self.image_dirs = image_dirs
        self.transform = transform
        self.target_size = target_size
        self.pad_size = pad_size  # UNet을 위해 패딩 후의 크기
        
    def __len__(self):
        return len(self.image_dirs)
    
    def __getitem__(self, idx):
        img_path = self.image_dirs[idx]
        image = Image.open(img_path).convert("RGB")
        
        # 이미지를 리사이즈하고 패딩 추가 (UNet에 맞게 352x544)
        resized_image = transforms.functional.resize(image, self.target_size)
        padding = (0, 0, self.pad_size[1] - self.target_size[1], self.pad_size[0] - self.target_size[0])
        padded_image = transforms.functional.pad(resized_image, padding, fill=0)  # 검정색 패딩 추가

        # 이미지 변환 적용
        if self.transform:
            padded_image = self.transform(padded_image)

        return padded_image, img_path  # 이미지와 경로를 함께 반환

In [27]:
test_ds = TestDataset(tst_dirs, transform=transform)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

imgs, path = next(iter(test_loader))
print(imgs.shape, len(path))

torch.Size([32, 3, 352, 544]) 32


In [ ]:
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image

# # 학습된 모델 불러오기 및 평가 모드로 설정
# # unet = torch.load('best_model.pth')  # 모델 경로를 지정
# unet.eval()  # 평가 모드로 설정
# # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # unet.to(device)

# # 예측 수행 및 시각화
# for i, images in enumerate(test_loader):
#     images = images.to(device)

#     # 모델 예측
#     with torch.no_grad():
#         outputs = unet(images)  # (B, num_classes, 352, 544)의 출력

#     # 패딩된 결과에서 패딩 제거 (350x525로 복원)
#     outputs = outputs[:, :, :350, :525]

#     # 각 클래스별로 마스크를 생성 (여기서는 0.5를 임계값으로 설정)
#     predicted_masks = (outputs > 0.5).float()  # 확률값이 0.5를 넘는 부분을 1로 설정 (임계값 조정 가능)

# #     # 각 클래스별로 마스크 시각화
# #     for class_idx in range(predicted_masks.shape[1]):  # 클래스마다 마스크 시각화
# #         mask = predicted_masks[0, class_idx].cpu().numpy()  # 첫 번째 배치의 클래스별 마스크

# #         plt.imshow(mask, cmap='gray')
# #         plt.title(f'Predicted Mask - Class {class_idx + 1}')
# #         plt.show()

# #         # 결과 저장 (예시)
# #         Image.fromarray((mask * 255).astype(np.uint8)).save(f'predicted_mask_class_{class_idx+1}_{i+1}.png')

In [28]:
import pandas as pd
import numpy as np
import os

# Run Length Encoding 함수 정의
def rle_encoding(mask):
    pixels = mask.flatten()  # 2D 마스크를 1D로 변환
    use_pixels = np.where(pixels == 1)[0]  # 1이 있는 위치를 찾음 (1-based indexing)
    
    if len(use_pixels) == 0:
        return ''  # 마스크가 없으면 빈 문자열 반환
    
#     print(use_pixels)
    # RLE 인코딩
#     rle = []
#     prev = use_pixels[0]
#     length = 1

    rle = []
    i = 0
    length = 0
    start = use_pixels[0]
    while True:
        if i == len(use_pixels) - 1:
            rle.extend([start, length])
            break
        else:
            if use_pixels[i] + 1 == use_pixels[i + 1]:
                i += 1
                length += 1
            else:
                rle.extend([start, length])
                length = 0
                start = use_pixels[i + 1]
                i += 1
            
#     for i in range(1, len(use_pixels)):
#         # 현재 인덱스가 이전 인덱스보다 정확히 1 크다면 연속된 영역
#         if use_pixels[i] == prev + 1:
#             length += 1
#         else:
#             # 연속된 영역이 끝나면 결과에 추가
#             rle.extend([prev - length + 1, length])  # 시작점과 길이 추가
#             prev = use_pixels[i]
#             length = 1  # 길이 초기화
    
    # 마지막 구간 추가
#     rle.extend([start, length])

    return ' '.join(map(str, rle))

In [29]:
# 2. 저장된 가중치 불러오기
unet.load_state_dict(torch.load('best_model.pth'))  # 모델 가중치 불러오기

# 3. 모델을 평가 모드로 설정
unet.eval()

# 4. 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
unet.to(device)

/tmp/ipykernel_36/3531311150.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load('best_model.pth'))  # 모델 가중치 불러오기


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [30]:
# 클래스 이름 정의
class_names = ['Fish', 'Flower', 'Gravel', 'Sugar']

# 결과를 저장할 리스트
submission = []

# 예측된 마스크 처리
for images, paths in tqdm(test_loader):  # 이미지와 경로를 함께 받음
    images = images.to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = unet(images)  # (B, num_classes, 352, 544)의 출력

    # 패딩된 결과에서 패딩 제거 (350x525로 복원)
    outputs = outputs[:, :, :350, :525]

    # 각 배치의 이미지 처리
    for batch_idx in range(outputs.size(0)):
        image_name = paths[batch_idx].split('/')[-1]  # 각 이미지의 이름 추출
        for class_idx in range(len(class_names)):
            class_name = class_names[class_idx]
            mask = (outputs[batch_idx, class_idx] > 0.5).float().cpu().numpy()  # 클래스별로 마스크 생성 (임계값 0.5)

            # RLE 인코딩 적용
            encoded_pixels = rle_encoding(mask)
#             print(encoded_pixels)
            # Image_Label 생성
            image_label = f'{image_name}_{class_name}'

            # 리스트에 추가
            submission.append({'Image_Label': image_label, 'EncodedPixels': encoded_pixels})

# 결과를 DataFrame으로 변환
submission_df = pd.DataFrame(submission)

  0%|          | 0/116 [00:00<?, ?it/s]

In [31]:
submission_df

,Image_Label,EncodedPixels
0,cb6c11f.jpg_Fish,155 337 680 337 1203 341 1726 344 2251 345 277...
1,cb6c11f.jpg_Flower,
2,cb6c11f.jpg_Gravel,
3,cb6c11f.jpg_Sugar,34868 4 35384 20 35905 27 36428 31 36950 34 37...
4,efe623a.jpg_Fish,
...,...,...
14787,f814066.jpg_Sugar,5 141 529 144 1054 144 1578 146 2103 147 2628 ...
14788,51473ad.jpg_Fish,5 335 530 337 1054 343 1579 344 2104 345 2629 ...
14789,51473ad.jpg_Flower,342 100 863 105 1384 110 1908 112 2432 114 295...
14790,51473ad.jpg_Gravel,


In [32]:
# CSV 파일로 저장
submission_df.to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 샘플 데이터를 가져오기
dataset = SegmentationDataset(train_label_df, train_list, transform=None)  # transform이 없는 버전으로 생성
image, masks = dataset[0]  # 예시로 첫 번째 이미지를 가져옴

# 마스크의 첫 번째 클래스를 시각화
mask_np = masks[0].numpy()  # 마스크의 첫 번째 클래스를 NumPy로 변환

# 마스크 이미지 출력
plt.imshow(mask_np, cmap='gray')
plt.title("Class 1 Mask")
plt.show()

# 원본 이미지도 시각화ㅡ
image_np = np.array(image)  # PIL 이미지를 NumPy로 변환
plt.imshow(image_np)
plt.title("Original Image")
plt.show()